# Gender Data Indicators
Generic Framework for extracting Gender Data

## World Bank API Usage Documentation
  
**API Documentation:** https://datahelpdesk.worldbank.org/knowledgebase/articles/898581</br>
**Base URL:** <a href="api.worldbank.org/v2/">api.worldbank.org/v2/</a></br>
**Parameters:** </br>

| Parameter | Description | Example |
| --- | --- | --- |
| `date` | Date or date range by year, month or quarter | `date=2000:2010` |
| `format` | Output format: xml, json, jsonP | `format=json` |
| `downloadformat` | Download format: csv, xml, excel | `downloadformat=csv` |
| `page` | Page number of the result set | `page=2` |
| `per_page` | Number of results per page (default 50) | `per_page=25` |
| `mrv` | Most recent values based on the number specified | `mrv=5` |
| `mrnev` | Most recent non-empty values based on the number specified | `mrnev=5` |
| `gapfill` | Fills values by back tracking to the next available period (works with mrv) | `gapfill=Y` |
| `frequency` | Frequency of values: Q (quarterly), M (monthly), Y (yearly) (works with mrv) | `frequency=M` |
| `footnote` | Fetches footnote detail in data calls | `footnote=y` |
| `language` | Local language translations for some countries | `language=vi` |


In [0]:
pip install pycountry

In [0]:
# Imports
import pandas as pd # Dataframe
import json # Parsing json to object
import requests # Making HTTP get requests
import pycountry

In [0]:
# List available countries
for country in pycountry.countries:
    print(country)

In [0]:
# List of countries
countries = [
    "Botswana", "Cameroon", "Gabon", "Gambia", "Ghana", "Kenya",
    "Eswatini", "Lesotho", "Malawi", "Mauritius", "Mozambique",
    "Namibia", "Nigeria", "Rwanda", "Seychelles", "Sierra Leone",
    "South Africa", "Togo", "Uganda", "Tanzania, United Republic of", "Zambia",
    "Bangladesh", "Brunei Darussalam", "India", "Malaysia", "Maldives",
    "Pakistan", "Singapore", "Sri Lanka", "Antigua and Barbuda", "Bahamas",
    "Barbados", "Belize", "Canada", "Dominica", "Grenada", "Guyana",
    "Jamaica", "Saint Lucia", "Saint Kitts and Nevis", "Saint Vincent and The Grenadines",
    "Trinidad and Tobago", "Cyprus", "Malta", "United Kingdom", "Australia",
    "Fiji", "Kiribati", "Nauru", "New Zealand", "Papua New Guinea", "Samoa",
    "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu"
]
print(len(countries))
# Dictionary to hold country names and their ISO codes
country_iso_codes = {}

for country in countries:
    try:
        iso_code = pycountry.countries.get(name=country).alpha_3 # Trinidad and Tobago | Trinidad & Tobago | trinidad and tobago [TTO] - https://www.iban.com/country-codes
        country_iso_codes[country] = iso_code
    except AttributeError:
        print(f"ISO code not found for {country}")



In [0]:
# IMPLEMENT A FILTER TO COMMONWEALTH COUNTRIES - https://thecommonwealth.org/our-member-countries

# Downloads the indicator using World Bank data API
def download_indicator(indicator_name):
  index = 0
  page = 1
  # Create a blank dataframe with the required columns
  df = pd.DataFrame(columns=["country_region", "iso_code", "year", "value", "unit"])
  # API returns data in pages, continue until all the data pages are loaded
  while True:
    # Built the GET url for the indicator data
    # Getting 1000 rows per page
    # Use null values for missing data (gapfill=N)
    url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_name}?format=json&per_page=1000&gapfill=N&page={page}" # REPLACE ALL WITH VARIABLE FOR ISO3_CODES
    response = requests.get(url)
    # Check if the GET request was successful else raise an exception
    if response.status_code == 200:
      # Convert the returned json to an object
      data = json.loads(response.text)
      # Get the total available pages
      total_pages = data[0]["pages"]
      print(f"Processing {indicator_name}: Page {page} / {total_pages}")
      # Loop through the available indicator values and add them to the dataframe
      for item in data[1]:
        df.loc[index] = { "country_region":item["country"]["value"], "iso_code": item["countryiso3code"], "year": item["date"], "value": item["value"], "unit": item["unit"] }
        index = index + 1
      # Exit loop if all the available pages have been read, else read the next page
      if (page > total_pages):
        break
      else:
        page = page + 1
    else:
      raise Exception(f"Failed to download indicator {indicator_name}")
  return df

In [0]:
# # Gets the name of the output file from indicator name
# def get_filename(indicator_name):
#   result = ""
#   for ch in indicator_name:
#     if ch == "." or ch == " ":
#       result += "_"
#     else:
#       result += ch
#   return f"{result}.csv"

# LOAD TO A DAFRAME

In [0]:
# # Downloads and saves the indicator to CSV file
# def save_indicator(indicator_name):
#   df = download_indicator(indicator_name)

#   # TODO: Save the dataframe to the database here.

#   # --- Uncomment the below lines to save to a CSV file ---
#   #out_file_path = f"indicators/{get_filename(indicator_name)}"
#   #print(f"Saving {indicator_name} to {out_file_path}")
#   #df.to_csv(out_file_path)
#   print(f"{indicator_name} download completed.")

In [0]:
# # Download the required indicators
# save_indicator("FIN21.T.D.2017.1")
# save_indicator("FIN21.T.D.2017.2")
# save_indicator("FIN21.T.D.2017")
# save_indicator("SG.GEN.PARL.ZS")
# save_indicator("SG.GEN.MNST.ZS")
# save_indicator("SE.SEC.ENRR.FE")
# save_indicator("UIS.FGP.5T8.F600")
# save_indicator("SL.TLF.CACT.FE.ZS")
# save_indicator("SG.LAW.NODC.HR")
# save_indicator("SG.OWN.LDAL.FE.ZS")
# save_indicator("SG.OPN.BANK.EQ")
# save_indicator("SG.CNT.SIGN.EQ")
# save_indicator("SP.DYN.SMAM.FE")
# save_indicator("SP.DYN.SMAM.MA")
# save_indicator("SP.M15.2024.FE.ZS")
# save_indicator("SP.M18.2024.FE.ZS")
# save_indicator("SG.VAW.1549.ME.ZS")
# save_indicator("SG.VAW.15PL.ME.ZS")
# save_indicator("SG.VAW.1549.LT.ME.ZS")
# save_indicator("SG.VAW.15PL.LT.ME.ZS")
# save_indicator("SG.LEG.DVAW")
# save_indicator("SH.STA.MMRT")
# save_indicator("SH.STA.MMRT.NE")
# save_indicator("SP.DYN.LE00.FE.IN")
# save_indicator("SP.DYN.LE00.MA.IN")
# save_indicator("SP.DYN.LE00.IN")